In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import os
import mne
import torch.nn.functional as F
import datetime

In [3]:
def get_epochs_and_labels(fif_file):
    # Load your EEG data
    raw = mne.io.read_raw_fif(fif_file, preload=True)
    
    # Get the events from the annotations
    events, _ = mne.events_from_annotations(raw)

    # T1 is left hand, T2 is right hand
    event_id = {'T1': 2, 'T2': 3}

    # Epochs start 0s before the trigger and end 0.5s after
    epochs = mne.Epochs(raw, events, event_id, tmin=0, tmax=0.5, baseline=None, preload=True)

    # Get the labels of the epochs
    labels = epochs.events[:, -1]

    # Change the labels to 5 and 6
    labels[labels == 2] = 0
    labels[labels == 3] = 1
    
    return epochs, labels

In [4]:
def combine_labels_from_folder(folder_path):
    # Create an empty list to store labels from each file
    all_labels = []
    
    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".fif"):
            file_path = os.path.join(folder_path, filename)
            _, labels = get_epochs_and_labels(file_path)
            
            all_labels.append(labels)
    
    # Concatenate all labels into one array
    combined_labels = np.concatenate(all_labels)
    
    return combined_labels

In [4]:
# Suppress long output
mne.set_log_level('ERROR')

# Load tensor data
tensor_path = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\openvibe\interim\openvibe_dataset_2024-06-08_21-56-50.pt"
tensors = torch.load(tensor_path)

# # Load labels
# labels_path = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\external\interim"
# labels_arr = combine_labels_from_folder(labels_path)

In [7]:
# Convert to PyTorch tensors


# Create a dataset and dataloaders
dataset = tensors
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [7]:
class EEG_MI_CNN(nn.Module):
    def __init__(self):
        super(EEG_MI_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.dropout = nn.Dropout(p=0.5)
        
        # Adjusted based on the final output size after conv layers
        self.fc1 = nn.Linear(64 * 2 * 15, 128)  
        self.fc2 = nn.Linear(128, 2)  # Assuming 2 classes for binary classification

    def _forward_conv_layers(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        return x

    def forward(self, x):
        x = self._forward_conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [8]:
class Advanced_EEG_MI_CNN(nn.Module):
    def __init__(self):
        super(Advanced_EEG_MI_CNN, self).__init__()
        # First convolutional layer with Batch Normalization and ReLU activation
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=(3, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Second convolutional layer with Batch Normalization and ReLU activation
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Third convolutional layer with Batch Normalization and ReLU activation
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(p=0.5)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 2 * 15, 256)  # Adjust based on final output size after conv layers
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)  # Assuming 2 classes for binary classification

    def _forward_conv_layers(self, x):
        # Forward pass through the first convolutional layer
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        # Forward pass through the second convolutional layer
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        # Forward pass through the third convolutional layer
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        return x

    def forward(self, x):
        # Forward pass through the convolutional layers
        x = self._forward_conv_layers(x)
        # Flatten the tensor to feed into fully connected layers
        x = x.view(x.size(0), -1)
        # Forward pass through the first fully connected layer with ReLU and dropout
        x = self.dropout(F.relu(self.fc1(x)))
        # Forward pass through the second fully connected layer with ReLU
        x = F.relu(self.fc2(x))
        # Output layer
        x = self.fc3(x)
        return x

In [10]:
class Wavelet_CNN(nn.Module):
    def __init__(self):
        super(Wavelet_CNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=64, kernel_size=(3, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.dropout = nn.Dropout(p=0.5)
        
        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Conv2d(512, 128, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(128, 512, kernel_size=1),
            nn.Sigmoid()
        )
        
        # Fully connected layers
        self.fc1 = nn.Linear(512 * 2 * 15, 128)  # Adjusted based on the final output size after conv layers
        self.fc2 = nn.Linear(128, 2)  # Assuming 2 classes for binary classification

    def _forward_conv_layers(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        
        # Attention mechanism
        attention_weights = self.attention(x)
        x = x * attention_weights
        
        return x

    def forward(self, x):
        x = self._forward_conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


In [8]:
class Advanced_EEG_MI_CNN_LSTM_Transformer(nn.Module):
    def __init__(self):
        super(Advanced_EEG_MI_CNN_LSTM_Transformer, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=(3, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size=128*2*15, hidden_size=128, num_layers=1, batch_first=True)
        
        # Transformer layer
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=128, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)
        
    def _forward_conv_layers(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        return x

    def forward(self, x):
        batch_size = x.size(0)
        # Forward pass through convolutional layers
        x = self._forward_conv_layers(x)
        
        # Flatten for LSTM
        x = x.view(batch_size, -1, 128*2*15)
        
        # Forward pass through LSTM
        x, (hn, cn) = self.lstm(x)
        
        # Transformer encoding
        x = self.transformer_encoder(x)
        
        # Take the output from the last sequence step for classification
        x = x[:, -1, :]
        
        # Forward pass through fully connected layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [9]:
# Save the model with date and time in the filename
def save_model(model, path):
    torch.save(model.state_dict(), path)

# Example usage:
model = Advanced_EEG_MI_CNN_LSTM_Transformer()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = correct / total
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Save the model with current date and time
current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
model_dir = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\models"
print("Model directory:", model_dir)
os.makedirs(model_dir, exist_ok=True)
model_filename = os.path.join(model_dir, f'{model.__class__.__name__}_{current_time}.pth')
print("Model filename:", model_filename)
save_model(model, model_filename)
print(f'Model saved as {model_filename}')

c:\Users\chakr\Anaconda3\envs\eeg_thesis\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/50, Loss: 0.6816
Validation Loss: 0.7401, Validation Accuracy: 0.4000
Epoch 2/50, Loss: 0.6768
Validation Loss: 0.7642, Validation Accuracy: 0.4000
Epoch 3/50, Loss: 0.5898
Validation Loss: 0.7421, Validation Accuracy: 0.4000
Epoch 4/50, Loss: 0.4480
Validation Loss: 0.7777, Validation Accuracy: 0.5000
Epoch 5/50, Loss: 0.2804
Validation Loss: 0.9768, Validation Accuracy: 0.5000
Epoch 6/50, Loss: 0.1658
Validation Loss: 1.1617, Validation Accuracy: 0.5000
Epoch 7/50, Loss: 0.0980
Validation Loss: 1.3967, Validation Accuracy: 0.5000
Epoch 8/50, Loss: 0.0511
Validation Loss: 1.7006, Validation Accuracy: 0.5000
Epoch 9/50, Loss: 0.0284
Validation Loss: 2.0218, Validation Accuracy: 0.5000
Epoch 10/50, Loss: 0.0142
Validation Loss: 2.3307, Validation Accuracy: 0.5000
Epoch 11/50, Loss: 0.0079
Validation Loss: 2.6304, Validation Accuracy: 0.5000
Epoch 12/50, Loss: 0.0044
Validation Loss: 2.9025, Validation Accuracy: 0.5000
Epoch 13/50, Loss: 0.0027
Validation Loss: 3.1424, Validation

In [12]:
# Define the model architecture (should match the original model)
model = Advanced_EEG_MI_CNN()

# Load the saved model's state dictionary
saved_model_path = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\models\Advanced_EEG_MI_CNN_2024-06-06_16-08-31.pth"
model.load_state_dict(torch.load(saved_model_path))

# Print all the parameters of the model
for name, param in model.named_parameters():
    print(name, param.size())
    

conv1.weight torch.Size([32, 8, 3, 3])
conv1.bias torch.Size([32])
bn1.weight torch.Size([32])
bn1.bias torch.Size([32])
conv2.weight torch.Size([64, 32, 3, 3])
conv2.bias torch.Size([64])
bn2.weight torch.Size([64])
bn2.bias torch.Size([64])
conv3.weight torch.Size([128, 64, 3, 3])
conv3.bias torch.Size([128])
bn3.weight torch.Size([128])
bn3.bias torch.Size([128])
fc1.weight torch.Size([256, 3840])
fc1.bias torch.Size([256])
fc2.weight torch.Size([64, 256])
fc2.bias torch.Size([64])
fc3.weight torch.Size([2, 64])
fc3.bias torch.Size([2])


In [13]:
# Load model and classify unseen data
def classify_data(model, data):
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
    return predicted